In [1]:
import torch
from torchvision.datasets import FashionMNIST
fm_train = FashionMNIST(root='.', train=True, download=True)
fm_test = FashionMNIST(root='.', train=False, download=True)

100.0%
100.0%
100.0%
100.0%


In [3]:
train_input = fm_train.data
train_target = fm_train.targets

In [4]:
# 정규화 - 0~1
train_scaled = train_input / 255.0

In [5]:
# 훈련세트, 검증 세트 
from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [6]:
# 모델을 위한 층 구성 
import torch.nn as nn

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 100),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(100, 10)
)

In [7]:
# GPU, CPU 결정, 모델에 연결 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 맥 - device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=100, bias=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): Linear(in_features=100, out_features=10, bias=True)
)

In [9]:
# 손실 함수, 옵티마이저 
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 훈련 하기 
train_hist = [] # 훈련세트 에포크 마다의 loss 값
val_hist = [] # 검증세트 에포크마다의 loss값
patience = 3 
best_loss = -1
early_stopping_counter = 0
epochs = 50 
